In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.feature_selection import RFE
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import EarlyStopping
import spacy
import time

# Initialize spaCy model
nlp = spacy.load('en_core_web_md')

# Function to embed text columns using spaCy
def embed_column_spacy(df, column):
    embeddings = df[column].apply(lambda x: nlp(x).vector if isinstance(x, str) else np.zeros(nlp.vocab.vectors_length))
    embedded_df = pd.DataFrame(embeddings.tolist(), index=df.index).add_prefix(f'{column}_embed_')
    return embedded_df

# Timing function
def timeit(method):
    def timed(*args, **kwargs):
        start_time = time.time()
        result = method(*args, **kwargs)
        end_time = time.time()
        print(f"{method.__name__} took {end_time - start_time:.2f} seconds")
        return result
    return timed

# Define neural network model
def create_model(optimizer='adam', dropout_rate=0.0, neurons=32):
    model = Sequential()
    model.add(Dense(neurons, input_dim=input_dim, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(neurons, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='linear'))
    model.compile(loss='mean_squared_error', optimizer=optimizer)
    return model

@timeit
def main():
    print("Loading data...")
    df = pd.read_csv('movies.csv')

    label = "Popularity"

    # Identify non-numeric columns
    non_numeric_columns = [col for col in df.columns if df[col].dtype == 'object' and col != label]

    # Embed text columns
    for col in non_numeric_columns:
        embedded_df = embed_column_spacy(df, col)
        df = pd.concat([df, embedded_df], axis=1)
    df.drop(columns=non_numeric_columns, inplace=True)

    # Handle missing values
    df = df.fillna(df.mean(numeric_only=True))

    # Splitting data
    X = df.drop(columns=[label])
    y = df[label]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Standardize the data
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    global input_dim
    input_dim = X_train.shape[1]

    # Neural Network Model
    model = KerasRegressor(build_fn=create_model, verbose=1)
    early_stopping = EarlyStopping(monitor='val_loss', patience=10)

    # Hyperparameter tuning
    param_grid = {
        'optimizer': ['adam', 'rmsprop'],
        'dropout_rate': [0.0, 0.1, 0.2],
        'neurons': [32, 64, 128],
        'batch_size': [10, 20, 50],
        'epochs': [50, 100]
    }

    grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error', n_jobs=-1)
    grid_result = grid.fit(X_train, y_train, validation_split=0.2, callbacks=[early_stopping])

    # Best parameters and model
    best_params = grid_result.best_params_
    print(f"Best parameters found: {best_params}")
    best_model = grid_result.best_estimator_

    # Evaluate model on test set
    y_pred = best_model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f"\nNeural Network Model Performance:")
    print(f"Mean Squared Error: {mse}")
    print(f"R^2 Score: {r2}")

    # Evaluate other models (optional, already included previously)
    ...

# Run the main function
main()


Loading data...


/var/folders/mv/1wspd8ss21x8drphht5pcly80000gn/T/ipykernel_2375/700758298.py:79: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasRegressor(build_fn=create_model, verbose=1)


Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50


2024-05-23 21:35:30.876347: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp_10.
2024-05-23 21:35:30.890874: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp_10.
2024-05-23 21:35:30.896317: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp_10.
2024-05-23 21:35:30.934001: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp_10.
2024-05-23 21:35:30.945512: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp_10.
2024-05-23 21:35:30.947257: E tensorflow/core/grappler/

82/82 [==============================] - 7s 34ms/step - loss: 64670.6016 - val_loss: 104227.2500
Epoch 2/50
82/82 [==============================] - 7s 33ms/step - loss: 64951.9062 - val_loss: 103913.7344
Epoch 2/50
82/82 [==============================] - 7s 33ms/step - loss: 64725.4648 - val_loss: 103355.8438
Epoch 2/50
82/82 [==============================] - 7s 34ms/step - loss: 64887.7461 - val_loss: 103561.0703
Epoch 2/50
83/83 [==============================] - 7s 50ms/step - loss: 65040.1992 - val_loss: 38095.3398
Epoch 2/50
83/83 [==============================] - 8s 51ms/step - loss: 65463.5820 - val_loss: 38802.0859
Epoch 2/50
82/82 [==============================] - 2s 22ms/step - loss: 63104.4922 - val_loss: 101767.1172
Epoch 3/50
83/83 [==============================] - 9s 52ms/step - loss: 70723.7188 - val_loss: 103317.6953
Epoch 2/50
82/82 [==============================] - 2s 25ms/step - loss: 63741.8867 - val_loss: 102973.2344
Epoch 3/50
83/83 [=======================

...

2024-05-23 22:15:29.907858: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp_10.


17/17 [==============================] - 2s 127ms/step - loss: 68534.7266 - val_loss: 34840.2461
Epoch 99/100
Epoch 35/50
17/17 [==============================] - ETA: 0s - loss: 75009.8438 

libc++abi: terminating due to uncaught exception of type std::__1::system_error: recursive_mutex lock failed: Invalid argument


17/17 [==============================] - 2s 110ms/step - loss: 61556.3945 - val_loss: 89128.4297
Epoch 38/50
17/17 [==============================] - 2s 125ms/step - loss: 75009.8438 - val_loss: 98840.2734
Epoch 26/50
17/17 [==============================] - 3s 167ms/step - loss: 64678.7383 - val_loss: 38179.2227
Epoch 3/50
17/17 [==============================] - 2s 133ms/step - loss: 65243.1992 - val_loss: 30658.0371
Epoch 36/50
17/17 [==============================] - 2s 119ms/step - loss: 66612.9219 - val_loss: 37045.3242
Epoch 11/50
17/17 [==============================] - 1s 75ms/step - loss: 49484.6680 - val_loss: 84543.8125
Epoch 48/50
17/17 [==============================] - 3s 153ms/step - loss: 69609.4141 - val_loss: 103231.4766
Epoch 4/50
 8/17 [=============>................] - ETA: 0s - loss: 38791.0234

TerminatedWorkerError: A worker process managed by the executor was unexpectedly terminated. This could be caused by a segmentation fault while calling the function or by an excessive memory usage causing the Operating System to kill the worker.

The exit codes of the workers are {SIGILL(-4)}